In [1]:
import pandas

In [2]:
import numpy as np

In [3]:
from pyiron import Project

In [4]:
from pyiron.atomistics.structure.atoms import Atoms

In [5]:
from pyiron.lammps.potential import LammpsPotentialFile

In [6]:
from mendeleev import get_table

In [7]:
from ase.data import reference_states, atomic_numbers

In [8]:
ptable = get_table('elements')

In [9]:
element_lst = ptable.symbol.values.tolist()

In [10]:
potential_selector = LammpsPotentialFile()

In [11]:
element_potential_dict = {}
for el in element_lst:
    df_pot = potential_selector.find(el)
    element_potential_dict[el] = np.unique(df_pot[df_pot['Model'] == 'NISTiprpy']['Name'].values).tolist()

In [12]:
not_working_potential_lst = [
    '2015--Islam-M-M--Li-S--LAMMPS--ipr1',
    '2011--Zhou-X-W--Li-Na-K-Rb-Cs-F-Cl-Br-I--LAMMPS--ipr1',
    '2015--Choudhary-K--Al--LAMMPS--ipr1',
    '2015--Kumar-A--Al-Ni--LAMMPS--ipr1',
    '2015--Kumar-A--Al-Ni-O--LAMMPS--ipr1',
    '2016--Zhou-X-W--Al-Cu--LAMMPS--ipr1',
    '2018--Zhou-X-W--Al-Cu-H--LAMMPS--ipr1',
    '2016--Zhang-P--Ti-O--LAMMPS--ipr1',
    '2016--Zhang-P--Ti-O--LAMMPS--ipr2',
    '2003--Mendelev-M-I--Fe-2--LAMMPS--ipr2',
    '2015--Zhou-X-W--C-Cu--LAMMPS--ipr1',
    '2015--Zhou-X-W--Cu-H--LAMMPS--ipr1',
    '2012--Ward-D-K--Cd-Te-Zn--LAMMPS--ipr1',
    '2013--Ward-D-K--Cd-Te-Zn--LAMMPS--ipr1',
    '2014--Zhou-X-W--Cd-Te-Se--LAMMPS--ipr1',
    '2012--Ward-D-K--Cd-Te--LAMMPS--ipr1',
    '2015--Broqvist-P--Ce-O--LAMMPS--ipr1',
    '2015--Thompson-A-P--Ta--LAMMPS--ipr1',
]

In [13]:
element_potential_dict

{'H': ['1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1',
  '2008--Chenoweth-K--C-H-O--LAMMPS--ipr1',
  '2008--Zhou-X-W--Pd-H--LAMMPS--ipr1',
  '2013--Hale-L-M--Pd-Ag-H-Hybrid--LAMMPS--ipr1',
  '2013--Hale-L-M--Pd-Ag-H-Morse--LAMMPS--ipr1',
  '2014--Bonny-G--W-H-He-1--LAMMPS--ipr1',
  '2014--Bonny-G--W-H-He-2--LAMMPS--ipr1',
  '2014--Nouranian-S--CH--ipr1',
  '2015--Zhou-X-W--Cu-H--LAMMPS--ipr1',
  '2018--Smirnova-D-E--Mg-H--LAMMPS--ipr1',
  '2018--Zhou-X-W--Al-Cu-H--LAMMPS--ipr1'],
 'He': ['2014--Bonny-G--W-H-He-1--LAMMPS--ipr1',
  '2014--Bonny-G--W-H-He-2--LAMMPS--ipr1'],
 'Li': ['2011--Zhou-X-W--Li-Na-K-Rb-Cs-F-Cl-Br-I--LAMMPS--ipr1',
  '2015--Islam-M-M--Li-S--LAMMPS--ipr1',
  '2016--Nichol-A--Li--LAMMPS--ipr1',
  '2016--Nichol-A--Li--LAMMPS--ipr2'],
 'Be': ['2013--Agrawal-A--Be--LAMMPS--ipr1',
  '2018--Byggmastar-J--Be-O--LAMMPS--ipr1'],
 'B': ['2012--Kinaci-A--B-N-C--LAMMPS--ipr1',
  '2017--Los-J-H--B-N--LAMMPS--ipr1'],
 'C': ['1989--Tersoff-J--Si-C--LAMMPS--ipr1',
  '1990--Tersoff-J--Si-

In [14]:
pot_lst = []
for k, v in element_potential_dict.items():
    for p in v:
        pot_lst.append(p)

In [15]:
len(pot_lst), len(np.unique(pot_lst)), len(not_working_potential_lst)

(474, 286, 18)

In [16]:
pr = Project('cohesive')

In [17]:
pr.remove_jobs(recursive=True)

This group does not exist in the HDF5 file lmp_bulk
This group does not exist in the HDF5 file lmp_vac


In [18]:
def setup_interactive_job(pr, job_name):
    job = pr.create_job(pr.job_type.Lammps, job_name)
    job.server.run_mode.interactive = True
    job.interactive_enforce_structure_reset = True
    return job

In [19]:
def calc_bulk(potential_lst, element, pr, size=5, job_bulk=None):
    if job_bulk is None:
        close_job = True
        job_bulk = setup_interactive_job(pr=pr, job_name='lmp_bulk') 
        job_bulk.calc_minimize(pressure=0.0)
    else: 
        close_job = False
    structure_bulk = pr.create_ase_bulk(element, cubic=True)
    structure_bulk.set_repeat([size, size, size])
    job_bulk.structure = structure_bulk.copy()
    for pot in potential_lst: 
        if pot not in not_working_potential_lst:
            job_bulk.potential = pot
            job_bulk.run()
    energy_lst = (job_bulk.output.energy_tot / len(structure_bulk)).tolist()
    if close_job:
        job_bulk.interactive_close()
    return energy_lst

In [20]:
def calc_vac(potential_lst, element, pr, job_vac=None, size=50.):
    if job_vac is None:
        close_job = True
        job_vac = setup_interactive_job(pr=pr, job_name='lmp_vac') 
    else: 
        close_job = False
    structure_vac = Atoms(element, positions=[[0., 0., 0.]], cell=[[size, 0., 0.], [0., size, 0.], [0., 0., size]])
    job_vac.structure = structure_vac.copy()
    for pot in potential_lst: 
        if pot not in not_working_potential_lst:
            job_vac.potential = pot
            job_vac.run()
    energy_lst = job_vac.output.energy_tot.tolist()
    if close_job:
        job_vac.interactive_close()
    return energy_lst

In [21]:
job_vac = setup_interactive_job(pr=pr, job_name='lmp_vac') 

In [22]:
job_bulk = setup_interactive_job(pr=pr, job_name='lmp_bulk') 
job_bulk.calc_minimize(pressure=0.0)

In [23]:
element_lst = []
potential_lst = []
bulk_energy_lst = []
vac_energy_lst = []
for k, v in element_potential_dict.items(): 
    ref = reference_states[atomic_numbers[k]]
    if ref is not None and ref['symmetry'] in ['bcc', 'hcp', 'fcc'] and len(v) != 0:
        bulk_energy = calc_bulk(
            potential_lst=v, 
            element=k, 
            pr=pr, 
            size=5,
            job_bulk=job_bulk,
        )[-len(v):]
        vac_energy = calc_vac(
            potential_lst=v, 
            element=k, 
            pr=pr,
            job_vac=job_vac,
        )[-len(v):]
        for p, b, z in zip(v, bulk_energy, vac_energy):
            element_lst.append(k)
            potential_lst.append(p)
            bulk_energy_lst.append(b)
            vac_energy_lst.append(z)

The job lmp_bulk was saved and received the ID: 4146403
The job lmp_vac was saved and received the ID: 4146404
dataset:  1 pair_coeff * * eam/alloy FeCr_d.eam.alloy Fe Cr
dataset:  2 pair_coeff * * eam/fs FeCr_s.eam.fs Fe Cr
dataset:  1 pair_coeff * * eam/alloy FeCrW_d.eam.alloy Fe Cr W
dataset:  2 pair_coeff * * eam/fs FeCrW_s.eam.fs Fe Cr W
dataset:  1 pair_coeff * * eam/alloy FeCrW_d.eam.alloy Fe Cr W
dataset:  2 pair_coeff * * eam/fs FeCrW_s.eam.fs Fe Cr W
dataset:  1 pair_coeff * * eam/alloy FeCr_d.eam.alloy Fe Cr
dataset:  2 pair_coeff * * eam/fs FeCr_s.eam.fs Fe Cr
dataset:  1 pair_coeff * * eam/alloy FeCrW_d.eam.alloy Fe Cr W
dataset:  2 pair_coeff * * eam/fs FeCrW_s.eam.fs Fe Cr W
dataset:  1 pair_coeff * * eam/alloy FeCrW_d.eam.alloy Fe Cr W
dataset:  2 pair_coeff * * eam/fs FeCrW_s.eam.fs Fe Cr W
dataset:  1 pair_coeff * * eam/alloy FeCr_d.eam.alloy Fe Cr
dataset:  2 pair_coeff * * eam/fs FeCr_s.eam.fs Fe Cr
dataset:  1 pair_coeff * * eam/alloy FeCrW_d.eam.alloy Fe Cr W
data

In [24]:
job_bulk.interactive_close()
job_vac.interactive_close()

In [25]:
df = pandas.DataFrame({'element': element_lst, 'potential': potential_lst, 'bulk': bulk_energy_lst, 'vac': vac_energy_lst})
df

,element,potential,bulk,vac
0,Li,2011--Zhou-X-W--Li-Na-K-Rb-Cs-F-Cl-Br-I--LAMMPS--ipr1,-1.647664,0.000000e+00
1,Li,2015--Islam-M-M--Li-S--LAMMPS--ipr1,-1.648000,0.000000e+00
2,Be,2013--Agrawal-A--Be--LAMMPS--ipr1,-3.225693,1.740616e+04
3,Be,2018--Byggmastar-J--Be-O--LAMMPS--ipr1,-3.623898,0.000000e+00
4,Na,2011--Zhou-X-W--Li-Na-K-Rb-Cs-F-Cl-Br-I--LAMMPS--ipr1,-3.623898,0.000000e+00
...,...,...,...,...
361,Pb,2003--Hoyt-J-J--Cu-Pb--LAMMPS--ipr1,-2.029999,-4.163336e-16
362,Pb,2004--Zhou-X-W--Pb--LAMMPS--ipr1,-2.019996,1.013279e-06
363,Pb,2004--Zhou-X-W--Pb--LAMMPS--ipr2,-2.019996,0.000000e+00
364,Pb,2018--Etesami-S-A--Pb-Sn--LAMMPS--ipr1,-2.040000,0.000000e+00
